<a href="https://colab.research.google.com/github/sirrelll/AVD/blob/main/Checkpoint_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("/content/goldstock v1.csv")

df

,Unnamed: 0,Date,Close,Volume,Open,High,Low
0,0,2024-01-19,2029.3,166078.0,2027.4,2041.9,2022.2
1,1,2024-01-18,2021.6,167013.0,2009.1,2025.6,2007.7
2,2,2024-01-17,2006.5,245194.0,2031.7,2036.1,2004.6
3,3,2024-01-16,2030.2,277995.0,2053.4,2062.8,2027.6
4,4,2024-01-12,2051.6,250946.0,2033.2,2067.3,2033.1
...,...,...,...,...,...,...,...
2506,2528,2014-01-28,1250.5,81426.0,1254.9,1261.9,1248.0
2507,2529,2014-01-27,1263.5,63419.0,1269.9,1280.1,1252.0
2508,2530,2014-01-24,1264.5,34998.0,1264.3,1273.2,1256.9
2509,2531,2014-01-23,1262.5,41697.0,1235.1,1267.1,1230.8


# **Data Cleaning**

#### Melihat Missing Values


Menangani missing values memiliki tujuan utama untuk meningkatkan kualitas serta keakuratan data dalam analisis. Dan juga banyak banyak algoritma dalam machine learning tidak dapat berfungsi dengan baik jika terdapat data yang kosong, sehingga perlu dilakukan imputasi atau penghapusan nilai yang hilang agar analisis lebih optimal.

In [6]:
print((df.isna().sum() / len(df)) * 100)

Unnamed: 0    0.0
Date          0.0
Close/Last    0.0
Volume        0.0
Open          0.0
High          0.0
Low           0.0
dtype: float64


Setelah dicek data ini tidak memiliki Missing Values.

# Duplicated Values

Duplicated values dapat mengurangi efisiensi pemroresan,jika didalam suatu data terdapat banyak duplikasi atau data yang sama maka hasil analisis bisa menjadi tidak akurat. Dan dengan cara menghapus semua duplikasi akan membuat hasil analisis menjadi lebih akurat.

In [7]:
df[df.duplicated()]

,Unnamed: 0,Date,Close/Last,Volume,Open,High,Low


Data ini tidak memiliki data yang muncul lebih dari satu kali atau tidak memiliki Duplicated Values.

# **Outliers**

Outliers ini bertujuan untuk meningkatkan keakuratan analisis data dan memastikan bahwa hasil yang diperoleh tidak tepat akibat nilai yang ekstrem.

Pengecekan Outliers

In [9]:
results = []

cols = df.select_dtypes(include=['float64', 'int64'])

for col in cols:
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5*iqr
  upper_bound = q3 + 1.5*iqr
  outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
  percent_outliers = (len(outliers)/len(df))*100
  results.append({'Kolom': col, 'Persentase Outliers': percent_outliers})

# Dataframe dari list hasil
results_df = pd.DataFrame(results)
results_df.set_index('Kolom', inplace=True)
results_df = results_df.rename_axis(None, axis=0).rename_axis('Kolom', axis=1)

# Tampilkan dataframe
display(results_df)

Kolom,Persentase Outliers
Unnamed: 0,0.000000
Close,0.000000
Volume,2.947033
Open,0.000000
High,0.000000
Low,0.000000


In [10]:
df

,Unnamed: 0,Date,Close,Volume,Open,High,Low
0,0,2024-01-19,2029.3,166078.0,2027.4,2041.9,2022.2
1,1,2024-01-18,2021.6,167013.0,2009.1,2025.6,2007.7
2,2,2024-01-17,2006.5,245194.0,2031.7,2036.1,2004.6
3,3,2024-01-16,2030.2,277995.0,2053.4,2062.8,2027.6
4,4,2024-01-12,2051.6,250946.0,2033.2,2067.3,2033.1
...,...,...,...,...,...,...,...
2506,2528,2014-01-28,1250.5,81426.0,1254.9,1261.9,1248.0
2507,2529,2014-01-27,1263.5,63419.0,1269.9,1280.1,1252.0
2508,2530,2014-01-24,1264.5,34998.0,1264.3,1273.2,1256.9
2509,2531,2014-01-23,1262.5,41697.0,1235.1,1267.1,1230.8


Terdapat Outliers "Volume" dan akan dilakukan imputasi menggunakan metode Winsorizing yang tetap mempertahankan struktur data tanpa membuang nilai ekstrem

In [11]:
columns_to_impute = ["Volume"]

for col in columns_to_impute:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Menggunakan .loc[] agar tidak muncul SettingWithCopyWarning
    df.loc[:, col] = df[col].clip(lower=lower_bound, upper=upper_bound)


Dicek Kembali

In [12]:
results = []

cols = df.select_dtypes(include=['float64', 'int64'])

for col in cols:
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5*iqr
  upper_bound = q3 + 1.5*iqr
  outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
  percent_outliers = (len(outliers)/len(df))*100
  results.append({'Kolom': col, 'Persentase Outliers': percent_outliers})

# Dataframe dari list hasil
results_df = pd.DataFrame(results)
results_df.set_index('Kolom', inplace=True)
results_df = results_df.rename_axis(None, axis=0).rename_axis('Kolom', axis=1)

# Tampilkan dataframe
display(results_df)

Kolom,Persentase Outliers
Unnamed: 0,0.0
Close,0.0
Volume,0.0
Open,0.0
High,0.0
Low,0.0


In [13]:
df

,Unnamed: 0,Date,Close,Volume,Open,High,Low
0,0,2024-01-19,2029.3,166078.0,2027.4,2041.9,2022.2
1,1,2024-01-18,2021.6,167013.0,2009.1,2025.6,2007.7
2,2,2024-01-17,2006.5,245194.0,2031.7,2036.1,2004.6
3,3,2024-01-16,2030.2,277995.0,2053.4,2062.8,2027.6
4,4,2024-01-12,2051.6,250946.0,2033.2,2067.3,2033.1
...,...,...,...,...,...,...,...
2506,2528,2014-01-28,1250.5,81426.0,1254.9,1261.9,1248.0
2507,2529,2014-01-27,1263.5,63419.0,1269.9,1280.1,1252.0
2508,2530,2014-01-24,1264.5,34998.0,1264.3,1273.2,1256.9
2509,2531,2014-01-23,1262.5,41697.0,1235.1,1267.1,1230.8


# **Inconsistent Values**

In [19]:
df

,Unnamed: 0,Date,Close,Volume,Open,High,Low
0,0,2024-01-19,2029.3,166078.0,2027.4,2041.9,2022.2
1,1,2024-01-18,2021.6,167013.0,2009.1,2025.6,2007.7
2,2,2024-01-17,2006.5,245194.0,2031.7,2036.1,2004.6
3,3,2024-01-16,2030.2,277995.0,2053.4,2062.8,2027.6
4,4,2024-01-12,2051.6,250946.0,2033.2,2067.3,2033.1
...,...,...,...,...,...,...,...
2506,2528,2014-01-28,1250.5,81426.0,1254.9,1261.9,1248.0
2507,2529,2014-01-27,1263.5,63419.0,1269.9,1280.1,1252.0
2508,2530,2014-01-24,1264.5,34998.0,1264.3,1273.2,1256.9
2509,2531,2014-01-23,1262.5,41697.0,1235.1,1267.1,1230.8


Setelah Melakukan Pengecakan data ini tidak memiliki Inconsistent Values, semua memiliki values nya masing masing. Karena ini data tentang stok emas tentunya pasti memiliki data yang konsisten.

# **Construct Data**

In [20]:
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")

df["Day"] = df["Date"].dt.day_name()

df["Daily_Price_Change"] = df["Close"] - df["Open"]

print(df.head())

   Unnamed: 0       Date   Close    Volume    Open    High     Low        Day  \
0           0 2024-01-19  2029.3  166078.0  2027.4  2041.9  2022.2     Friday   
1           1 2024-01-18  2021.6  167013.0  2009.1  2025.6  2007.7   Thursday   
2           2 2024-01-17  2006.5  245194.0  2031.7  2036.1  2004.6  Wednesday   
3           3 2024-01-16  2030.2  277995.0  2053.4  2062.8  2027.6    Tuesday   
4           4 2024-01-12  2051.6  250946.0  2033.2  2067.3  2033.1     Friday   

   Daily_Price_Change  
0                 1.9  
1                12.5  
2               -25.2  
3               -23.2  
4                18.4  


In [21]:
df

,Unnamed: 0,Date,Close,Volume,Open,High,Low,Day,Daily_Price_Change
0,0,2024-01-19,2029.3,166078.0,2027.4,2041.9,2022.2,Friday,1.9
1,1,2024-01-18,2021.6,167013.0,2009.1,2025.6,2007.7,Thursday,12.5
2,2,2024-01-17,2006.5,245194.0,2031.7,2036.1,2004.6,Wednesday,-25.2
3,3,2024-01-16,2030.2,277995.0,2053.4,2062.8,2027.6,Tuesday,-23.2
4,4,2024-01-12,2051.6,250946.0,2033.2,2067.3,2033.1,Friday,18.4
...,...,...,...,...,...,...,...,...,...
2506,2528,2014-01-28,1250.5,81426.0,1254.9,1261.9,1248.0,Tuesday,-4.4
2507,2529,2014-01-27,1263.5,63419.0,1269.9,1280.1,1252.0,Monday,-6.4
2508,2530,2014-01-24,1264.5,34998.0,1264.3,1273.2,1256.9,Friday,0.2
2509,2531,2014-01-23,1262.5,41697.0,1235.1,1267.1,1230.8,Thursday,27.4


Berdasarkan studi kasus yang saya ambil yaitu menganalisis tren pasar dari emas, disini saya menambahkan daily_price_change atau harga emas yang berganti keseharian tentunya bisa membuat hasil analisis lebih tepat dan akurat.

# **Data Reduction**

In [22]:
df = df.drop('Day', axis=1)

In [23]:
df

,Unnamed: 0,Date,Close,Volume,Open,High,Low,Daily_Price_Change
0,0,2024-01-19,2029.3,166078.0,2027.4,2041.9,2022.2,1.9
1,1,2024-01-18,2021.6,167013.0,2009.1,2025.6,2007.7,12.5
2,2,2024-01-17,2006.5,245194.0,2031.7,2036.1,2004.6,-25.2
3,3,2024-01-16,2030.2,277995.0,2053.4,2062.8,2027.6,-23.2
4,4,2024-01-12,2051.6,250946.0,2033.2,2067.3,2033.1,18.4
...,...,...,...,...,...,...,...,...
2506,2528,2014-01-28,1250.5,81426.0,1254.9,1261.9,1248.0,-4.4
2507,2529,2014-01-27,1263.5,63419.0,1269.9,1280.1,1252.0,-6.4
2508,2530,2014-01-24,1264.5,34998.0,1264.3,1273.2,1256.9,0.2
2509,2531,2014-01-23,1262.5,41697.0,1235.1,1267.1,1230.8,27.4


Disini saya merasa kolom "Day" tidak terlalu relevan untuk analisis tren data ini.